In [108]:
from langgraph.graph import StateGraph, START, END 
from typing_extensions import TypedDict
from langgraph.types import CachePolicy
from langgraph.cache.memory import InMemoryCache
from datetime import datetime

In [116]:
class State(TypedDict):
    time: str
    
graph_builder = StateGraph(State)

In [117]:
def node_one(state: State):
    return {}
    
def node_two(state: State):
    return {"time": f"{datetime.now()}"}

def node_three(state: State):
    return {}
    

In [118]:
graph_builder.add_node("node_one", node_one)
graph_builder.add_node("node_two", node_two, cache_policy=CachePolicy(ttl=20))
graph_builder.add_node("node_three", node_three)

graph_builder.add_edge(START, "node_one")
graph_builder.add_edge("node_one", "node_two")
graph_builder.add_edge("node_two", "node_three")
graph_builder.add_edge("node_three", END)

In [119]:
graph = graph_builder.compile(cache=InMemoryCache())
print(graph.get_graph().draw_ascii())

+-----------+  
| __start__ |  
+-----------+  
       *       
       *       
       *       
 +----------+  
 | node_one |  
 +----------+  
       *       
       *       
       *       
 +----------+  
 | node_two |  
 +----------+  
       *       
       *       
       *       
+------------+ 
| node_three | 
+------------+ 
       *       
       *       
       *       
  +---------+  
  | __end__ |  
  +---------+  


In [120]:
import time
print(graph.invoke({}))
time.sleep(5)
print(graph.invoke({}))
time.sleep(5)
print(graph.invoke({}))
time.sleep(5)
print(graph.invoke({}))
time.sleep(5)
print(graph.invoke({}))
time.sleep(5)
print(graph.invoke({}))
time.sleep(5)
print(graph.invoke({}))


{'time': '2025-09-28 19:44:57.443243'}
{'time': '2025-09-28 19:44:57.443243'}
{'time': '2025-09-28 19:44:57.443243'}
{'time': '2025-09-28 19:44:57.443243'}
{'time': '2025-09-28 19:45:17.477594'}
{'time': '2025-09-28 19:45:17.477594'}
{'time': '2025-09-28 19:45:17.477594'}
